In [ ]:
# run, reading in files
import pandas as pd
import numpy as np
oura_2lag = pd.read_csv("C:/Users/rocke/Downloads/oura_stress_data/oura_stress_ptsd_ace_2lag.csv")
X = oura_2lag.loc[:, oura_2lag.columns != "daily_stressed"]
y = oura_2lag[["daily_stressed"]]

In [ ]:
# ignore this
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, 
                                                    random_state = 10, stratify = X["participant_id"])

In [ ]:
# run to get sum of pcl
pcl_sum = []
for i in range(len(oura_2lag)):
    pcl = 0
    for j in range(1, 18):
        pcl += oura_2lag["pcl_" + str(j)][i]  # just a clever way to get the sum of all pcl columns
    pcl_sum.append(pcl)
pcl_sum
oura_2lag["pcl_sum"] = pcl_sum  # adding in column

In [ ]:
# run to get sum of ace
ace_sum = []
for i in range(len(oura_2lag)):
    ace = 0
    for j in range(1, 11):
        ace += oura_2lag["ace_" + str(j)][i]  # just a clever way to get teh sum of all ace columns
    ace_sum.append(ace)
ace_sum
oura_2lag["ace_sum"] = ace_sum  # adding in column

In [ ]:
# ignore, just testing out smf
import statsmodels.formula.api as smf
log_reg = smf.logit(formula = "daily_stressed ~ pcl_1", data = oura_2lag).fit_regularized(method = "l1")
print(log_reg.summary())

In [ ]:
# run. standardizes and splits into training and testing data. return 3 objects. summary, confusion matrix, and accuracy
import statsmodels.formula.api as smf
from sklearn.metrics import (confusion_matrix, accuracy_score, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split
def logit_ftn(predictors, target, data1):
    copy = data1.copy()  # making a copy of the original dataframe. don't want to mess with original data
    empty = ""  # empty string used later for formula argument inside logit()
    predictor_list = []  # empty list for predictors for subsetting
    for i in predictors:
        predictor_list.append(i)
    for i in predictor_list:
        copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
    for i in predictors:
        empty += i + " + "
    empty = empty[:-3]  # erases the " + "
    eqn = target + "~" + empty  # formula equation
    log_reg = smf.logit(formula = eqn, data = copy).fit()
    predictor_list.append("participant_id")  # adding this here now because it was not necessary to scale participant_id
    X = copy.loc[:, predictor_list]
    y = copy[["daily_stressed"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, 
                                                    random_state = 10, stratify = X["participant_id"])
    yhat = log_reg.predict(X_test)  # getting the prediction values
    prediction = []
    for i in yhat:
        if i <= 0.3:
            prediction.append(0)
        else:
            prediction.append(1)
    cm = confusion_matrix(y_test, prediction)  # confustion matrix
    accuracy = accuracy_score(y_test, prediction)  # gives the accuracy on testing data
    return log_reg.summary(), cm, accuracy  # returns a list of variables

In [ ]:
# running to see if it works
summary, cm, accuracy = logit_ftn(["duration", "pcl_1", "pcl_2", "pcl_3"], "daily_stressed", oura_2lag)

In [ ]:
# checking out the model
print(summary)
print(cm)
print(accuracy)

In [ ]:
# input a threshold start, stop, and step interval... as well as predictors, target variable, and dataframe. returns
# threshold value, confusion matrix, and plots ROC curve

# wanted to find an "optimal" threshold value. it is important to note that for different scenarios/goals, "optimal" may 
# have different definitions. revelation came thanks to Dr. Franks. 
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
def best_acc_model(start, stop, step, predictors, target, data1):
    rates = []
    for t in np.arange(start, stop, step):
        copy = data1.copy()  # making a copy of the original dataframe. don't want to mess with original data
        empty = ""  # empty string used later for formula argument inside logit()
        predictor_list = []  # empty list for predictors for subsetting
        for i in predictors:
            predictor_list.append(i)
        for i in predictor_list:
            copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
        for i in predictors:
            empty += i + " + "
        empty = empty[:-3]  # erases the " + "
        eqn = target + "~" + empty  # formula equation
        log_reg = smf.logit(formula = eqn, data = copy).fit()
        predictor_list.append("participant_id")  # adding this here now because it was not necessary to scale participant_id
        X = copy.loc[:, predictor_list]
        y = copy[["daily_stressed"]]
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, 
                                                        random_state = 10, stratify = X["participant_id"])
        yhat = log_reg.predict(X_test)  # getting the prediction values of probabilities
        fpr, tpr, _ = metrics.roc_curve(y_test,  yhat)
        prediction = []
        for i in yhat:
            if i <= t:
                prediction.append(0)
            else:
                prediction.append(1)
        cm = confusion_matrix(y_test, prediction)
        cm_list.append(cm)
        accuracy = accuracy_score(y_test, prediction)
        print("Threshold value:", t, "and accuracy:", accuracy)
        print(cm)
        plt.plot(fpr, tpr)

In [ ]:
# testing the function
best_acc_model(0.2, 0.9, 0.01, ["daily_shifts", "worked_with_covid", "restless", "score"], "daily_stressed", oura_2lag)

In [ ]:
# testing out my *first* function on this subset
summary, cm, accuracy = logit_ftn(["score", "daily_shifts", "worked_with_covid", "restless"], "daily_stressed", oura_2lag)

In [ ]:
print(summary)  # works fine

In [ ]:
print(accuracy)  # works fine

In [ ]:
oura_2lag["participant_id"].value_counts()

# could it be that a participant(s) is always stressed or never stressed? means that the variable will be perfectly
# correlated to the response. 

for i in oura_2lag["participant_id"].unique():
    yes = oura_2lag[oura_2lag["participant_id"] == i]["daily_stressed"].value_counts()
    if len(yes) == 1:
        print(i)
        
# there are some participants who are always stress/no stressed, found after running the above for loop.

In [ ]:
# just taking away these participants. allows the logit model to run
test = copy["participant_id"].isin(["U-5HZ8AMM58QDLR5JZV2NL", "U-ATEES286BB2TCX95R9T3", "U-JA3NXGB6CL1W7CZGH5MA", 
                                    "U-L8KA5ZY6HBEEQJ2W7C6V", "U-NCR7YC8J7R151B5XUHYT", "U-U9C8WE1MNK1SC43YZYMP",
                                    "U-URPG7EFR92D8CQP36S52", "U-Z6KKZKWG52EQAEDPPAVR"])
copy = copy[~test]
copy

# this block ended up being useless since I utilized regularization. this means coefficients won't blow up.

In [ ]:
test = copy["participant_id"].isin(["U-5HZ8AMM58QDLR5JZV2NL", "U-ATEES286BB2TCX95R9T3", "U-JA3NXGB6CL1W7CZGH5MA", 
                                    "U-L8KA5ZY6HBEEQJ2W7C6V", "U-NCR7YC8J7R151B5XUHYT", "U-U9C8WE1MNK1SC43YZYMP",
                                    "U-URPG7EFR92D8CQP36S52", "U-Z6KKZKWG52EQAEDPPAVR"])
copy = copy[~test]

predictor_standardize = ["restless", "duration", "score", "ace_sum", "pcl_sum"]
for i in predictor_standardize:
    copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
log_reg = smf.logit(formula = "daily_stressed ~ daily_shifts + worked_with_covid + score*restless + score*duration + pcl_sum + ace_sum", 
                    data = copy).fit()
print(log_reg.summary())
X = copy.loc[:, copy.columns != "daily_stressed"]
y = copy[["daily_stressed"]]
yhat = log_reg.predict(X)
fpr, tpr, _ = metrics.roc_curve(y,  yhat)
plt.plot(fpr, tpr)

In [ ]:
# WITHOUT participant id
copy = oura_2lag.copy()
'''test = copy["participant_id"].isin(["U-5HZ8AMM58QDLR5JZV2NL", "U-ATEES286BB2TCX95R9T3", "U-JA3NXGB6CL1W7CZGH5MA", 
                                    "U-L8KA5ZY6HBEEQJ2W7C6V", "U-NCR7YC8J7R151B5XUHYT", "U-U9C8WE1MNK1SC43YZYMP",
                                    "U-URPG7EFR92D8CQP36S52", "U-Z6KKZKWG52EQAEDPPAVR"])'''
#copy = copy[~test]
predictor_standardize = ["restless", "duration", "score", "ace_sum", "pcl_sum"]
for i in predictor_standardize:
    copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
log_reg = smf.logit(formula = "daily_stressed ~ daily_shifts + worked_with_covid + score*restless + score*duration + pcl_sum + ace_sum", 
                    data = copy).fit()
print(log_reg.summary())
X = copy.loc[:, copy.columns != "daily_stressed"]
y = copy[["daily_stressed"]]
yhat = log_reg.predict(X)
fpr, tpr, _ = metrics.roc_curve(y,  yhat)
plt.plot(fpr, tpr)

In [ ]:
# ONLY participant id
copy = oura_2lag.copy()
'''test = copy["participant_id"].isin(["U-5HZ8AMM58QDLR5JZV2NL", "U-ATEES286BB2TCX95R9T3", "U-JA3NXGB6CL1W7CZGH5MA", 
                                    "U-L8KA5ZY6HBEEQJ2W7C6V", "U-NCR7YC8J7R151B5XUHYT", "U-U9C8WE1MNK1SC43YZYMP",
                                    "U-URPG7EFR92D8CQP36S52", "U-Z6KKZKWG52EQAEDPPAVR"])'''
#copy = copy[~test]
predictor_standardize = ["restless", "duration", "score", "ace_sum", "pcl_sum"]
for i in predictor_standardize:
    copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
log_reg = smf.logit(formula = "daily_stressed ~ C(participant_id)", 
                    data = copy).fit_regularized(method = "l1")
print(log_reg.summary())
X = copy.loc[:, copy.columns != "daily_stressed"]
y = copy[["daily_stressed"]]
yhat = log_reg.predict(X)
fpr, tpr, _ = metrics.roc_curve(y,  yhat)
plt.plot(fpr, tpr)

In [ ]:
# look at how many days per subject. smaller numbers can affect their probability greatly

In [ ]:
sum(copy["pcl_sum"])/len(copy["pcl_sum"])

In [ ]:
# BOTH participant id and features
copy = oura_2lag.copy()
predictor_standardize = ["restless", "duration", "score", "pcl_sum", "ace_sum"]
for i in predictor_standardize:
    copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
log_reg = smf.logit(formula = "daily_stressed ~ C(participant_id) + daily_shifts + worked_with_covid + score*restless + score*duration", 
                    data = copy).fit_regularized(method = "l1")
print(log_reg.summary())
X = copy.loc[:, copy.columns != "daily_stressed"]
y = copy[["daily_stressed"]]
yhat = log_reg.predict(X)
fpr, tpr, _ = metrics.roc_curve(y,  yhat)
plt.plot(fpr, tpr)

# collinearity, brute force method. look up a package for multicollinearity. no reg. in mixed effects. might be because
# we don't care about coefficient in mixed effects

In [ ]:
# BOTH participant id and features
copy = oura_2lag.copy()
predictor_standardize = ["restless", "duration", "score", "pcl_sum", "ace_sum"]
for i in predictor_standardize:
    copy[i] = (copy[i] - copy[i].mean()) / copy[i].std()
print(copy)
'''log_reg = smf.mixedlm(formula = "daily_stressed ~ daily_shifts + worked_with_covid + score", 
                    data = copy, groups = copy["participant_id"]).fit(method=["lbfgs"])
print(log_reg.summary())
X = copy.loc[:, copy.columns != "daily_stressed"]
y = copy[["daily_stressed"]]
yhat = log_reg.predict(X)
fpr, tpr, _ = metrics.roc_curve(y,  yhat)
plt.plot(fpr, tpr)'''

In [ ]:
# maybe try plotting precision vs recall

***CAN IGNORE EVERYTHING TO THE BOTTOM OF THIS, JUST LOOKING AT SOME VARIABLES***

In [ ]:
pcl = X_train[X_train.columns[pd.Series(X_train.columns).str.startswith('pcl_')]]  # choosing all PTSD vars
ace = X_train[X_train.columns[pd.Series(X_train.columns).str.startswith('ace_')]]  # choosing all ACE vars
pcl_ace = pd.concat([pcl, ace], axis = 1)  # combining both PTSD and ACE vars
pcl_ace = pcl_ace.loc[:, pcl_ace.columns != "ace_ts"]  # dropping ace_ts
pcl_ace
X_train_pcl_ace = pcl_ace  # renaming (I don't follow this convention for other training sets later)

In [ ]:
import statsmodels.api as sm
log_reg = sm.Logit(y_train, X_train_pcl_ace).fit()  # fitting a logistic regression
print(log_reg.summary())

In [ ]:
# all of this is for the testing data to match training data parameters
pcl_test = X_test[X_test.columns[pd.Series(X_test.columns).str.startswith('pcl_')]] 
ace_test = X_test[X_test.columns[pd.Series(X_test.columns).str.startswith('ace_')]]
pcl_ace_test = pd.concat([pcl_test, ace_test], axis = 1)
pcl_ace_test = pcl_ace_test.loc[:, pcl_ace_test.columns != "ace_ts"]
X_test_pcl_ace = pcl_ace_test

In [ ]:
from sklearn.metrics import (confusion_matrix, accuracy_score, ConfusionMatrixDisplay)

# confusion matrix and performance matrix of ACE and PTSD vars
yhat = log_reg.predict(X_test_pcl_ace)
prediction = list(map(round, yhat))
cm = confusion_matrix(y_test, prediction)
print ("Confusion Matrix : \n", cm)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction))

In [ ]:
import matplotlib as plt

# confusion matrix with ACE and PTSD included
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

In [ ]:
# creating different subsets of variables to use
covid = X_train[["worked_with_covid"]]
sleep = X_train[["score"]]
covid_test = X_test[["worked_with_covid"]]
sleep_test = X_test[["score"]]
pcl_ace_covid = pd.concat([pcl_ace, covid], axis = 1)
pcl_ace_sleep = pd.concat([pcl_ace, sleep], axis = 1)
pcl_ace_covid_sleep = pd.concat([pcl_ace_covid, sleep], axis = 1)
covid_sleep = pd.concat([covid, sleep], axis = 1)

# testing portion
pcl_ace_covid_test = pd.concat([pcl_ace_test, covid_test], axis = 1)
pcl_ace_sleep_test = pd.concat([pcl_ace_test, sleep_test], axis = 1)
pcl_ace_covid_sleep_test = pd.concat([pcl_ace_covid_test, sleep_test], axis = 1)
covid_sleep_test = pd.concat([covid_test, sleep_test], axis = 1)

In [ ]:
# fitting some more models
log_reg2 = sm.Logit(y_train, pcl_ace_covid).fit()  # PTSD, ACE, and WORK w/ COVID
log_reg3 = sm.Logit(y_train, pcl_ace_sleep).fit()  # PTSD, ACE, AND SLEEP SCORE
log_reg4 = sm.Logit(y_train, pcl_ace_covid_sleep).fit()  # PTSD, ACE, WORK w/ COVID, AND SLEEP SCORE

In [ ]:
# PTSD, ACE, and WORK W/ COVID
yhat2 = log_reg2.predict(pcl_ace_covid_test)
prediction2 = list(map(round, yhat2))
cm2 = confusion_matrix(y_test, prediction2)
print ("Confusion Matrix : \n", cm2)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction2))

In [ ]:
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2)
disp2.plot()

In [ ]:
# PTSD, ACE, AND SLEEP SCORE
yhat3 = log_reg3.predict(pcl_ace_sleep_test)
prediction3 = list(map(round, yhat3))
cm3 = confusion_matrix(y_test, prediction3)
print ("Confusion Matrix : \n", cm3)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction3))

In [ ]:
disp3 = ConfusionMatrixDisplay(confusion_matrix=cm3)
disp3.plot()

In [ ]:
# PTSD, ACE, WORK w/ COVID, and SLEEP SCORE
yhat4 = log_reg4.predict(pcl_ace_covid_sleep_test)
prediction4 = list(map(round, yhat4))
cm4 = confusion_matrix(y_test, prediction4)
print ("Confusion Matrix : \n", cm4)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction4))

In [ ]:
disp4 = ConfusionMatrixDisplay(confusion_matrix=cm4)
disp4.plot()

In [ ]:
y_test.hist()

In [ ]:
# fitting more models
log_reg5 = sm.Logit(y_train, covid).fit()  # just work w/ covid
log_reg6 = sm.Logit(y_train, sleep).fit()  # just sleep score
log_reg7 = sm.Logit(y_train, covid_sleep).fit()  # just covid and sleep score

In [ ]:
# WORK w/ COVID
yhat5 = log_reg5.predict(covid_test)
prediction5 = list(map(round, yhat5))
cm5 = confusion_matrix(y_test, prediction5)
print ("Confusion Matrix : \n", cm5)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction5))

In [ ]:
# SLEEP SCORE
yhat6 = log_reg6.predict(sleep_test)
prediction6 = list(map(round, yhat6))
cm6 = confusion_matrix(y_test, prediction6)
print ("Confusion Matrix : \n", cm6)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction6))

In [ ]:
# WORK w/ COVID AND SLEEP SCORE
yhat7 = log_reg7.predict(covid_sleep_test)
prediction7 = list(map(round, yhat7))
cm7 = confusion_matrix(y_test, prediction7)
print ("Confusion Matrix : \n", cm7)
 
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction7))

In [ ]:
print(log_reg2.summary())
print(log_reg3.summary())
print(log_reg4.summary())

In [ ]:
print(log_reg5.summary())
print(log_reg6.summary())
print(log_reg7.summary())

In [ ]:
copy = oura_2lag.copy()
test = copy["participant_id"].isin(["U-5HZ8AMM58QDLR5JZV2NL", "U-ATEES286BB2TCX95R9T3", "U-JA3NXGB6CL1W7CZGH5MA", 
                                    "U-L8KA5ZY6HBEEQJ2W7C6V", "U-NCR7YC8J7R151B5XUHYT", "U-U9C8WE1MNK1SC43YZYMP",
                                    "U-URPG7EFR92D8CQP36S52", "U-Z6KKZKWG52EQAEDPPAVR"])
copy = copy[~test]

mixed_oura = smf.mixedlm("daily_stressed ~ worked_with_covid + daily_shifts + pcl_sum + ace_sum", oura_2lag, groups = oura_2lag["participant_id"])
mixed_oura_fit = mixed_oura.fit()
mixed_oura_fit.summary()

In [ ]:
oura_2lag["pcl_sum"].nunique()